# <span style="color:Maroon">Data Preparation - Part 4

<span style="color:Green">__Overview:__ Create dataset at county level by dividing the population by race type

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os as os

In [2]:
os.chdir("..")
cwd = os.getcwd()
other_data = "\\Data\\other data"
os.chdir(f'{cwd}{other_data}')

In [3]:
data = pd.read_csv("cc-est2013-alldata.csv",encoding = "ISO-8859-1")

In [4]:
data.shape

(358302, 80)

In [5]:
# Drop 2013 data, as 2013 is the lag year
data = data[data["YEAR"]<6]
data = data[data["YEAR"]!=2]
data = data[data["YEAR"]!=3]

In [6]:
states = list(np.unique(np.array(data["STATE"])))

In [7]:
varlist = ["SUMLEV", "STATE", "COUNTY", "CTYNAME", "YEAR", "STNAME"]

In [8]:
years = [1,4,5]

In [9]:
def transpose_derive(df,year,state,county):
    df = df.set_index("AGEGRP")
    df = df.transpose()
    df['PC_OF_TOTAL'] = df[0]/df[0].max()
    for i in range(1,19):
        df["PC"+str(i)+"_"+str(2010+year)] = df[i]/df[0]
    vars = list(df.columns)
    cols = vars[0:19]
    vars = vars[19:]
    cols = [(str(2010+year)+"_"+str(col)) for col in cols]
    vars = cols+vars
    df.columns = vars
    categories = list(dict.fromkeys(list(df.index)))
    out_data = pd.DataFrame()
    for i in range(0,len(categories)):
        df1 = df[df.index==categories[i]]
        cols = list(df1.columns)
        cols = [(categories[i]+str(col)) for col in cols]
        df1.columns = cols 
        df1.index = [state*1000+county]
        out_data =out_data.merge(df1, how="outer", left_index=True, right_index=True)
    return out_data

In [10]:
out_data = pd.DataFrame()
for i in range(0,len(states)):
    data1 = data[data["STATE"]==states[i]]
    county = list(np.unique(np.array(data1["COUNTY"])))
    for j in range(0,len(county)):
        data2 = data1[data1["COUNTY"]==county[j]]
        outp = pd.DataFrame()
        for k in range(0,len(years)):
            data3 = data2[data2["YEAR"]==years[k]]
            data3 = data3.drop(varlist,axis=1)
            tmp = transpose_derive(data3,k,states[i],county[j])
            outp = outp.merge(tmp, how="outer", left_index=True, right_index=True)
        out_data = out_data.append(outp)

In [11]:
out_data.shape

(3143, 8322)

In [12]:
out_data.head()

,TOT_POP2010_0,TOT_POP2010_1,TOT_POP2010_2,TOT_POP2010_3,TOT_POP2010_4,TOT_POP2010_5,TOT_POP2010_6,TOT_POP2010_7,TOT_POP2010_8,TOT_POP2010_9,...,HNAC_FEMALEPC9_2012,HNAC_FEMALEPC10_2012,HNAC_FEMALEPC11_2012,HNAC_FEMALEPC12_2012,HNAC_FEMALEPC13_2012,HNAC_FEMALEPC14_2012,HNAC_FEMALEPC15_2012,HNAC_FEMALEPC16_2012,HNAC_FEMALEPC17_2012,HNAC_FEMALEPC18_2012
1001,54571,3579,3991,4290,4290,3080,3157,3330,4157,4086,...,0.222222,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.0,0.0,0.0
1003,182265,11158,11599,11926,11600,9449,10247,10709,11558,11995,...,0.058824,0.000000,0.029412,0.000000,0.029412,0.058824,0.000000,0.0,0.0,0.0
1005,27457,1702,1642,1599,1731,1794,2010,1808,1819,1809,...,0.153846,0.307692,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1007,22915,1378,1405,1440,1543,1491,1603,1646,1747,1635,...,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1009,57322,3616,3880,4084,4033,3107,3436,3441,3892,4033,...,0.142857,0.000000,0.047619,0.000000,0.000000,0.000000,0.047619,0.0,0.0,0.0


In [13]:
out_data.to_csv(f"{cwd}\\Data\\Model data\\Population_Race.csv")